# Модели

In [ ]:
%pip install -q -U bitsandbytes
#accelerate gradio datasets

In [ ]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Основная модель Mistral 7B Instruct v0.2

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,                         
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    device_map="auto",
)

model_id = "/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1"
#

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(model_id)

## Модель классификатор DistilBERT 12 Classes

In [ ]:
from transformers import (
    pipeline,
    AutoModelForSequenceClassification,
)

classification_model_id = 'AlexxxSem/distilbert-12-classes'

classification_model = AutoModelForSequenceClassification.from_pretrained(classification_model_id)
classification_tokenizer = AutoTokenizer.from_pretrained(classification_model_id)

classify = pipeline("text-classification",
                    model=classification_model,
                    tokenizer=classification_tokenizer)

## Модель суммаризации DistilBart CNN 12 6 SamSum

In [ ]:
from transformers import BartForConditionalGeneration

summarization_model_id = 'philschmid/distilbart-cnn-12-6-samsum'
#sshleifer/distilbart-cnn-12-6

summarization_model = BartForConditionalGeneration.from_pretrained(summarization_model_id)
summarization_tokenizer = AutoTokenizer.from_pretrained(summarization_model_id)

summary = pipeline("summarization",
                   model=summarization_model,
                   tokenizer=summarization_tokenizer)

# Данные квеста

## Датасет со сценарием и логикой квеста

In [ ]:
import pandas as pd

data = [
 {'Step': 0,
  'Variant': 'YES',
  'Action': 'Heavy rain has started',
  'Question': 'Go forward or back home?',
  'ToStep': 1,
  'UserGame': False},
 {'Step': 0,
  'Variant': 'NO',
  'Action': 'Say something with variant of "Thank you, goodbye".',
  'Question': 'Maybe another time?',
  'ToStep': 5,
  'UserGame': False},
 {'Step': 0,
  'Variant': 'RPT',
  'Action': 'Profound misunderstanding',
  'Question': 'Do you want to continue?',
  'ToStep': 0,
  'UserGame': True},
 {'Step': 1,
  'Variant': 'FORWARD',
  'Action': 'to right way rabbit, to left way turtle',
  'Question': 'Go right or go left?',
  'ToStep': 2,
  'UserGame': False},
 {'Step': 1,
  'Variant': 'BACK',
  'Action': 'home umbrella',
  'Question': 'Go or stay?',
  'ToStep': 4,
  'UserGame': False},
 {'Step': 1,
  'Variant': 'RPT',
  'Action': 'cold thunder',
  'Question': 'Where?',
  'ToStep': 1,
  'UserGame': True},
 {'Step': 2,
  'Variant': 'RIGHT',
  'Action': 'rabbit carrot big house',
  'Question': 'How are you feeling?',
  'ToStep': 3,
  'UserGame': False},
 {'Step': 2,
  'Variant': 'LEFT',
  'Action': 'turtle lake small house',
  'Question': 'How are you feeling?',
  'ToStep': 3,
  'UserGame': False},
 {'Step': 2,
  'Variant': 'RPT',
  'Action': 'rabit with honey or turtle with jam?',
  'Question': 'rabbit or turtle?',
  'ToStep': 2,
  'UserGame': True},
 {'Step': 3,
  'Variant': 'HAPPY',
  'Action': 'We are very glad to make you fun see you next time',
  'Question': 'Want try again?',
  'ToStep': 5,
  'UserGame': False},
 {'Step': 3,
  'Variant': 'SAD',
  'Action': 'home see many friends and fun',
  'Question': 'Stay with us or go away?',
  'ToStep': 4,
  'UserGame': False},
 {'Step': 3,
  'Variant': 'RPT',
  'Action': 'snake bubble',
  'Question': 'How are you feeling?',
  'ToStep': 3,
  'UserGame': True},
 {'Step': 4,
  'Variant': 'GO',
  'Action': 'to right way rabbit, to left way turtle',
  'Question': 'Go right or go left?',
  'ToStep': 2,
  'UserGame': False},
 {'Step': 4,
  'Variant': 'STAY',
  'Action': 'more friends and fun',
  'Question': 'Can we see you again?',
  'ToStep': 5,
  'UserGame': False},
 {'Step': 4,
  'Variant': 'RPT',
  'Action': 'something beatyfull',
  'Question': 'Go or stay?',
  'ToStep': 4,
  'UserGame': True},
 {'Step': 5,
  'Variant': 'START',
  'Action': 'Траля-ля и Труля-ля. Смотреть ниже',
  'Question': 'Hello!',
  'ToStep': 0,
  'UserGame': False},
]

df = pd.DataFrame(data)
df

## Начальная сцена

In [ ]:
start_scene = f'''
    Write a fairy tail, consisting of 4-5 sentences, using next instruction:

    1. Use the characters and the environment below:
    ## Bear Boris, deep forest ##

    2. Add in story next:
    ## Birds ##

    3. Use in story next descriptions:
    ## very hungry ##

    4. Finish story with question:
    ## Do you want to continue? ##
'''

# Функции

## Стартовые параметры

In [ ]:
# Счетчики Шагов и Пользовательских Игр
step_count = 5 # не менять, так надо
user_count = 0

# Для хранения текущей Сцены
step_result = []

## Функция проверки начала квеста

In [ ]:
def start():
    global step_count, user_count
    
    if step_count == 5:
        step_count = 0
        user_count = 0
        
        return True
    
    else:
        return False

## Функция для обработки шага и пользовательской игры

In [ ]:
def quest_counter(to_step, user_game):
    global step_count
    global user_count
    
    # Меняем номер следующего шага
    step_count = to_step
    
    # Обработка в зависимости от значения user_count и to_step
    if user_count != 0 and to_step == step_count:
        user_count = 0
        step_count += 1
    elif user_count != 0 and to_step != step_count:
        user_count = 0
    
    # Включения счетчика в случае UserGame
    if user_game:
        user_count += 1

## Функция создание подготовленного промпта

In [ ]:
def use_prompt_template(action, query, question):
    # Суммаризация прошлой сцены
    last_step = summary(step_result)[0]['summary_text']
    
    step_prompt = f'''
    Write a compelling continuation for the story, crafting 4-5 sentences based on the following guidelines:

    1. Utilize the characters and surroundings introduced in the current narrative:
    ## {last_step} ##

    2. Integrate the storyline with the following action:
    ## {action} ##

    3. Incorporate vivid descriptions into the narrative, utilizing the provided details:
    ## {query} ##

    4. Conclude the story by introducing a thought-provoking question:
    ## {question} ##
    '''
    return step_prompt

## Функция для обработки сценария и пользовательского ввода

In [ ]:
def modify_query(query):
    # Проверяем на Начало
    if start():
        modified_query = start_scene

        return modified_query
    
    else:
        # Фильтруем DataFrame по текущему шагу
        step_df = df[df.Step == step_count]
        
        # Проверяем запрос пользователя с помощью модели классификации
        desired_value = classify(query)[0]['label']
        print('Class: 'classify(query)[0], end='\n\n') # Для тестирования
        
        # Ищем подходящий Вариант
        if desired_value in step_df.Variant.unique():
            selected_row = step_df[step_df.Variant == desired_value]
            
            # Присваиваем значения для варианта
            action = selected_row.Action.values[0]
            question = selected_row.Question.values[0]
            to_step = selected_row.ToStep.values[0]
            user_game = selected_row.UserGame.values[0]
            
        else: # Запускаем повтор для уточнения
            selected_row = step_df[step_df.Variant == 'RPT']
            
            # Присваиваем значения для повтора
            action = selected_row.Action.values[0]
            question = selected_row.Question.values[0]
            to_step = selected_row.ToStep.values[0]
            user_game = selected_row.UserGame.values[0]
            
    # Изменяем запрос с помощью функции создания промпта
    modified_query = use_prompt_template(action, query, question)
    
    # Меняем шаги с помощью функции обработки шагов
    quest_counter(to_step, user_game)
    
    print('Query:---------------', modified_query, end='\n') # Для тестирования
        
    return modified_query

## Функция для запроса и ответа модели

### Пример запроса-ответа

In [ ]:
sample_instruction = '''
    Write a compelling continuation for the story, crafting 4-5 sentences based on the following guidelines:

    1. Utilize the characters and surroundings introduced in the current narrative:
    ## In a cozy people's house, there lived a clever mouse named Mikey. One day, Mikey encountered a mischievous cat named Tom. Instead of the expected chase, they engaged in a series of funny games between them, surprising everyone with their unexpected friendship. The people in the house couldn't help but smile at the playful antics of the unlikely duo. As the sun set, casting a warm glow over the scene, a question lingered in the air, "Do you want to continue witnessing the amusing adventures of Mikey and Tom?" ##

    2. Integrate the storyline with the following action:
    ## Dog Charlie in the garden ##

    3. Incorporate vivid descriptions into the narrative, utilizing the provided details:
    ## go right ##

    4. Conclude the story by introducing a thought-provoking question:
    ## Do you Happy? ##
'''

sample_story = '''
    As the sun dipped below the horizon, casting a warm glow over the cozy people's house, Mikey and Tom found themselves facing a new adventure in the garden. Intrigued by the sudden appearance of Dog Charlie, they decided to go right, embarking on a journey filled with laughter and surprises. In the garden, the trio encountered colorful flowers, playful butterflies, and hidden pathways. The unlikely friendship between Mikey, Tom, and Charlie flourished amidst the beauty of nature. Now, as their delightful journey unfolded, a thought-provoking question hung in the air, "Do you Happy?" — a sentiment echoed by the joyous trio exploring the enchanting garden together.
'''

In [ ]:
import re

def generate_response(query):
    global step_result
    
    messages = [
        {"role": "user", "content": sample_instruction},
        {"role": "assistant", "content": sample_story},
        {"role": "user", "content": modify_query(query)},
    ]
    
    #messages = [{"role": "user", "content": modify_query(query, df)}]
    #prompt_text = f"<s>[INST]{sample_question}[/INST]{sample_answer}</s>[INST]{query}[/INST]"
    
    #input_encoding = tokenizer.encode_plus(prompt_text, return_tensors="pt")['input_ids'].to(device)
    input_encoding = tokenizer.apply_chat_template(messages, return_tensors="pt").to(device)

    generated_ids = model.generate(input_encoding,
                                   pad_token_id=tokenizer.eos_token_id,
                                   max_new_tokens=1000,
                                   do_sample=True)

    decoded_response = tokenizer.batch_decode(generated_ids,
                                              skip_special_tokens=True)

    # Обрезаем полный ответ, оставляем только сгенерированную историю
    step_result = re.sub(r'.*\[\/INST\]\n   ', '', decoded_response[0], flags=re.DOTALL)
        
    # Для тестирования
    print('-------------------', end='\n')
    print('To Step>>: ', step_count)
    print('User Game: ', user_count, end='\n\n')
    print("Story:-----------------", end='\n\n')

    return step_result

In [ ]:
# Пример использования
query = "right"
desired_value = 'NO'
modified_query = modify_query(query, df)

print("Modified Query:", modified_query)
print("Step_count:", step_count)
print("User_count:", user_count)

# Тестирование

In [ ]:
# Для Начала любое значение, дальше отвечать на вопросы как захочется
message_answer = 'smiling'

print('Current Step>>: ', step_count, end='\n')
      
# print(generate_response(message_answer))

In [ ]:
# Для быстрого сброса или установки нужных шагов

# step_count = 5
# user_count = 0
# step_count, user_count

# Для быстрой проверки или замены

#step_result = """"""
# step_result


In [ ]:
import gradio as gr

def response(message, history):
    return generate_response(message)

gr.ChatInterface(response).launch()

In [ ]:
!pip install -q -U datasets

In [ ]:
from datasets import Dataset

ds = Dataset.from_pandas(df)
ds[2]['Step'] = 2

In [ ]:
ds[2]['Step'] = 2

In [ ]:
df1 = ds.to_pandas()
df1

In [ ]:
ds[2]['Step']

In [ ]:
ds.save_to_disk('/kaggle/working/')

In [ ]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

access_token_read = UserSecretsClient().get_secret("HF_TOKEN")
login(token = access_token_read)